Se importan librerias

In [1]:
import pandas as pd
import json
import gzip
import nltk as nlt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nlt.download("vader_lexicon")
import re

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gerar\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Steam Games
Carga y tranformacion de los datos 

In [2]:
df_1 = []
with gzip.open('Json/steam_games.json.gz','rt', encoding='utf-8') as f:
    for line in f:
        df_1.append(json.loads(line))
steam_games  = pd.DataFrame(df_1)

In [3]:
steam_games.head() #visualizacion de los primeros 5 registros

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
steam_games.info #visualizacion de informacion de los datos

<bound method DataFrame.info of               publisher                                 genres  \
0                   NaN                                    NaN   
1                   NaN                                    NaN   
2                   NaN                                    NaN   
3                   NaN                                    NaN   
4                   NaN                                    NaN   
...                 ...                                    ...   
120440  Ghost_RUS Games  [Casual, Indie, Simulation, Strategy]   
120441           Sacada              [Casual, Indie, Strategy]   
120442     Laush Studio            [Indie, Racing, Simulation]   
120443         SIXNAILS                        [Casual, Indie]   
120444              NaN                                    NaN   

                        app_name                     title  \
0                            NaN                       NaN   
1                            NaN                   

In [5]:
#Elimina filas nulas

steam_games = steam_games.dropna(how='any', axis=0)
steam_games = steam_games.reset_index(drop=True)

In [6]:
steam_games.columns # Veo columna que voy a eliminar

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [7]:
steam_games.isnull().sum() #cuento los nulos de cada columa para ver como tratar los datos

publisher       0
genres          0
app_name        0
title           0
url             0
release_date    0
tags            0
reviews_url     0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

In [9]:
steam_games.sample(10) # extraigo varias columas para muestra de forma aleatoria para verificar otro tipo de nulos

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
12480,Degica,"[Design &amp; Illustration, Web Publishing]",RPG Maker MV - Karugamo Contemporary BGM Pack 02,RPG Maker MV - Karugamo Contemporary BGM Pack 02,http://store.steampowered.com/app/624850/RPG_M...,2017-04-20,"[Design & Illustration, Web Publishing]",http://steamcommunity.com/app/624850/reviews/?...,"[Downloadable Content, Steam Trading Cards, Pa...",9.99,False,624850,"KADOKAWA,Yoji Ojima"
2678,Might and Delight,"[Adventure, Casual, Indie]",Shelter 2,Shelter 2,http://store.steampowered.com/app/275100/Shelt...,2015-03-09,"[Survival, Open World, Adventure, Simulation, ...",http://steamcommunity.com/app/275100/reviews/?...,"[Single-player, Steam Achievements, Full contr...",14.99,False,275100,Might and Delight
21719,GSC World Publishing,[Strategy],Cossacks: Art of War,Cossacks: Art of War,http://store.steampowered.com/app/4870/Cossack...,2002-03-31,"[Strategy, RTS, Historical]",http://steamcommunity.com/app/4870/reviews/?br...,[Single-player],4.99,False,4870,GSC Game World
699,Square Enix,"[Action, RPG]",Dungeon Siege III: Treasures of the Sun,Dungeon Siege III: Treasures of the Sun,http://store.steampowered.com/app/39270/Dungeo...,2011-10-25,"[RPG, Action]",http://steamcommunity.com/app/39270/reviews/?b...,"[Single-player, Multi-player, Co-op, Downloada...",6.99,False,39270,Obsidian Entertainment
22351,Io-Interactive A/S,[Action],Hitman 2: Silent Assassin,Hitman 2: Silent Assassin,http://store.steampowered.com/app/6850/Hitman_...,2002-10-01,"[Stealth, Action, Third Person, Singleplayer, ...",http://steamcommunity.com/app/6850/reviews/?br...,[Single-player],6.99,False,6850,Io-Interactive A/S
8510,Arcade Injection,"[Casual, Indie, Racing, Sports, Early Access]",ThunderWheels,ThunderWheels,http://store.steampowered.com/app/296790/Thund...,2017-11-02,"[Early Access, Racing, Sports, Indie, Casual, ...",http://steamcommunity.com/app/296790/reviews/?...,"[Single-player, Multi-player, Local Multi-Play...",8.99,True,296790,Arcade Injection
17034,Artifex Mundi,"[Adventure, Casual]",The Secret Order 2: Masked Intent,The Secret Order 2: Masked Intent,http://store.steampowered.com/app/399890/The_S...,2015-11-05,"[Casual, Adventure, Hidden Object, Point & Cli...",http://steamcommunity.com/app/399890/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",9.99,False,399890,Sunward Games
13173,"Playdius, Plug In Digital","[Action, Indie]",Splasher,Splasher,http://store.steampowered.com/app/446840/Splas...,2017-02-07,"[Platformer, Indie, Action, Controller, Cartoo...",http://steamcommunity.com/app/446840/reviews/?...,"[Single-player, Steam Achievements, Full contr...",14.99,False,446840,Splashteam
21773,Paradox Interactive,[Simulation],Cities in Motion: Design Marvels,Cities in Motion: Design Marvels,http://store.steampowered.com/app/73013/Cities...,2011-05-20,[Simulation],http://steamcommunity.com/app/73013/reviews/?b...,"[Single-player, Downloadable Content, Steam Ac...",3.99,False,73013,Colossal Order Ltd.
13435,Cleiton Oliveira,"[Action, Casual, Indie]",Through the Mirror,Through the Mirror,http://store.steampowered.com/app/548720/Throu...,2017-01-07,"[Casual, Action, Indie]",http://steamcommunity.com/app/548720/reviews/?...,"[Single-player, Steam Achievements, Full contr...",1.99,False,548720,Cleiton Oliveira


In [10]:
df_games = steam_games.drop(['publisher','app_name','url','tags','reviews_url','specs','price','early_access'], axis=1) #Eliminacion de columas iniecesarias

Genero los dummy para la columna 'genres'

In [11]:
dummy_genres = pd.get_dummies(df_games['genres'].explode()) # Crear variables ficticias para cada género

dummy_genres = dummy_genres.groupby(dummy_genres.index).sum() # Agrupar por índice original y sumar para obtener variables ficticias por fila

steam_games = pd.concat([df_games, dummy_genres], axis=1) # Agregar las variables ficticias a la base de datos

steam_games = steam_games.drop(columns=['genres']) # Eliminar la columna original de géneros


Formateo la fecha para trabajar solo con la fecha de 'relase_date'

In [12]:
# Reemplazar los valores no válidos por NaN
steam_games["release_date"] = pd.to_datetime(steam_games["release_date"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos
df_filtered = steam_games.dropna(subset=["release_date"])
# Convertir la columna "release_date" a tipo datetime
df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])
# Rellenando los valores faltantes en las columna'release_date' con la moda
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    steam_games[col].fillna(steam_games[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'release_date'
steam_games['year'] = steam_games['release_date'].dt.year.astype('Int64')
steam_games.drop('release_date', axis=1, inplace=True)

C:\Users\gerar\AppData\Local\Temp\ipykernel_9508\1122611128.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])


# User Reviews
Carga y tranformacion de los datos 

In [16]:
df_2 = []
with gzip.open('Json/user_reviews.json.gz','rt', encoding='utf-8') as f:
    for line in f:
        df_2.append(eval(line))
user_reviews  = pd.DataFrame(df_2)

In [17]:
user_reviews.head() #visualizacion de los primeros 5 registros

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [18]:
user_reviews.info #visualizacion de informacion de los datos

<bound method DataFrame.info of                  user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
2              evcentric             http://steamcommunity.com/id/evcentric   
3                  doctr                 http://steamcommunity.com/id/doctr   
4              maplemage             http://steamcommunity.com/id/maplemage   
...                  ...                                                ...   
25794  76561198306599751  http://steamcommunity.com/profiles/76561198306...   
25795           Ghoustik              http://steamcommunity.com/id/Ghoustik   
25796  76561198310819422  http://steamcommunity.com/profiles/76561198310...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                   

In [20]:
user_reviews.isnull().sum() # Cuento los nulos de cada columa para ver como tratar los datos

user_id     0
user_url    0
reviews     0
dtype: int64

In [21]:
user_reviews.sample(10) # Extraigo varias columas para muestra de forma aleatoria para verificar otro tipo de nulos

,user_id,user_url,reviews
2877,hotsaucdo,http://steamcommunity.com/id/hotsaucdo,"[{'funny': '1 person found this review funny',..."
25077,76561198105130240,http://steamcommunity.com/profiles/76561198105...,"[{'funny': '', 'posted': 'Posted August 20, 20..."
13662,kronix129,http://steamcommunity.com/id/kronix129,"[{'funny': '3 people found this review funny',..."
14292,Lockouttaheaven,http://steamcommunity.com/id/Lockouttaheaven,"[{'funny': '', 'posted': 'Posted February 27, ..."
5582,76561198092339855,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted October 28, 2..."
18925,shadowmakenshi,http://steamcommunity.com/id/shadowmakenshi,"[{'funny': '', 'posted': 'Posted December 27, ..."
13066,76561198165671770,http://steamcommunity.com/profiles/76561198165...,"[{'funny': '', 'posted': 'Posted May 20, 2015...."
7937,stewysdsdsdsadasdgsadf,http://steamcommunity.com/id/stewysdsdsdsadasd...,"[{'funny': '', 'posted': 'Posted July 7, 2015...."
5704,cggamess123,http://steamcommunity.com/id/cggamess123,"[{'funny': '', 'posted': 'Posted July 2, 2014...."
9395,76561198089220459,http://steamcommunity.com/profiles/76561198089...,"[{'funny': '', 'posted': 'Posted October 1, 20..."


Desanidado

In [22]:
df_2 = user_reviews.explode('reviews') # creo un archivo con la columan que me interesa ingresar
dfreviewsOpen = pd.concat([df_2.drop(['reviews'], axis=1), df_2['reviews'].apply(pd.Series)], axis=1) #con esta expresion  creo un nuevo df donde esta la info desanidada en columnas
dfreviewsOpen

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D,NaN


In [23]:
dfreviewsOpen.columns # Veo todas las columnas para ver con cuales me quedo

Index([    'user_id',    'user_url',       'funny',      'posted',
       'last_edited',     'item_id',     'helpful',   'recommend',
            'review',             0],
      dtype='object')

In [24]:
# contar nulos por columnas para proximos tratamientos
dfreviewsOpen.isnull().sum()

user_id            0
user_url           0
funny             28
posted            28
last_edited       28
item_id           28
helpful           28
recommend         28
review            28
0              59333
dtype: int64

In [25]:
df_reviews = dfreviewsOpen.drop(['user_url','funny','last_edited','helpful',0], axis=1) #Eliminacion de columas iniecesarias

In [26]:
# eliminar la palabra posted de la columna posted

df_reviews['posted'] = df_reviews['posted'].str.replace('Posted', '')


Tratamiento para formatear las columnas 'posted'

In [27]:
# Reemplazar los valores no válidos por NaN
df_reviews["posted"] = pd.to_datetime(df_reviews["posted"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos
df_filtered = df_reviews.dropna(subset=["posted"])
# Convertir la columna "release_date" a tipo datetime
df_filtered["posted"] = pd.to_datetime(df_filtered["posted"])
# Rellenando los valores faltantes en las columna'release_date' con la moda
columns_fill_mode = ['posted']
for col in columns_fill_mode:
    df_reviews[col].fillna(df_reviews[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'release_date'
df_reviews['posted_year'] = df_reviews['posted'].dt.year.astype('Int64')
df_reviews.drop('posted', axis=1, inplace=True)

C:\Users\gerar\AppData\Local\Temp\ipykernel_9508\1528883759.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["posted"] = pd.to_datetime(df_filtered["posted"])


Tratamiento para el modelo

In [28]:
df_reviews['review'] = df_reviews['review'].astype('str') # Convertir el tipo de dato a string

In [29]:
df_reviews['review'] = df_reviews['review'].apply(lambda x: re.sub(r'[^a-zA-Z]', '', x.lower())) # Eliminar caracteres no válidos

In [30]:
#Incializar el modelo
sia = SentimentIntensityAnalyzer()

# Función para asignar valores según la escala
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Devuelve neutral si está vacía o NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2  # Bueno puntuacion
        elif compound_score <= -0.05:
            return 0  # Mala puntuacion
        else:
            return 1
    else:
        return 1  # Devuelve neutral para valores que no son cadenas
    


# Convertir la columna 'review' a tipo de dato str
    
df_reviews['review'] = df_reviews['review'].astype(str)

# Aplicar la función get_sentiment_score a la columna 'review'
df_reviews['sentiment_score'] = df_reviews['review'].apply(get_sentiment_score)

# eliminar la columna 'review'

df_reviews = df_reviews.drop('review', axis=1)

In [31]:
df_reviews.isnull().sum() # Cuento los nulos de cada columa para ver como tratar los datos

user_id             0
item_id            28
recommend          28
posted_year         0
sentiment_score     0
dtype: int64

In [32]:
#Elimina filas nulas

df_reviews = df_reviews.dropna(how='any', axis=0)
df_reviews = df_reviews.reset_index(drop=True)

In [33]:
df_reviews # Vista final

,user_id,item_id,recommend,posted_year,sentiment_score
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,2
...,...,...,...,...,...
59300,76561198312638244,70,True,2014,2
59301,76561198312638244,362890,True,2014,2
59302,LydiaMorley,273110,True,2014,2
59303,LydiaMorley,730,True,2014,2


# User Items
Carga y tranformacion de los datos 

In [34]:
df_3 = []
with gzip.open('Json/users_items.json.gz','rt', encoding='utf-8') as f:
    for line in f:
        df_3.append(eval(line))
users_items  = pd.DataFrame(df_3)

In [35]:
users_items.head() #visualizacion de los primeros 5 registros

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [36]:
df_items = users_items.drop(columns=['items_count','user_url']) #Eliminacion de columas iniecesarias

In [37]:
df_items.isnull().sum() # Cuento los nulos de cada columa para ver como tratar los datos

user_id     0
steam_id    0
items       0
dtype: int64

In [38]:
df_items2 = df_items.explode('items') # creo un archivo con la columan que me interesa ingresar
test = pd.concat([df_items2.drop(['items'], axis=1), df_items2['items'].apply(pd.Series)], axis=1) #con esta expresion  creo un nuevo df donde esta la info desanidada en columnas
test

In [ ]:
test.sample(10) # vextraigo varias columnas para muestra de forma 

In [ ]:
test.isnull().sum() # Veo la cantidad de nulos que hay en cada columna

user_id                   0
steam_id                  0
item_id               16806
item_name             16806
playtime_forever      16806
playtime_2weeks       16806
0                   5170015
dtype: int64

In [ ]:
test.isnull() # Extraccion de los nulos de cada columna para ver como tratar los datos

,user_id,steam_id,item_id,item_name,playtime_forever,playtime_2weeks,0
0,False,False,False,False,False,False,True
0,False,False,False,False,False,False,True
0,False,False,False,False,False,False,True
0,False,False,False,False,False,False,True
0,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...
88308,False,False,False,False,False,False,True
88308,False,False,False,False,False,False,True
88308,False,False,False,False,False,False,True
88308,False,False,False,False,False,False,True


In [ ]:
df_items = test.dropna(subset=["item_id"]) # Se crea un nuevo dataframe sin los nulos
df_items.isnull().sum() # Se verifica que no haya nulos

user_id                   0
steam_id                  0
item_id                   0
item_name                 0
playtime_forever          0
playtime_2weeks           0
0                   5153209
dtype: int64

In [ ]:
df_items = df_items.drop(columns=[0]) # Se elimina la columna 0
df_items = df_items.drop(columns=['steam_id']) # Se elimina la columna steam_id
df_items = df_items.drop(columns=['item_name']) # Se elimina la columna steam_id

In [43]:
df_items.duplicated().sum() # Cuento los duplicados 

0

In [44]:
df_items = df_items.drop_duplicates(subset=['user_id','item_id']) # Elimina los duplicados
df_items.duplicated().sum() # Verifica que no hayan duplicados

0

In [ ]:
df_items[df_items['playtime_forever'] == 0] # Extrer filas con valores 0 en la columna 'playtime_forever'

In [47]:
# Eliminar filas con valor 0 en la columna 'playtime_forever' ya que signifa que ese usuario no tiene tiempo de jugada

df_items = df_items[df_items['playtime_forever']!= 0]
df_items.sample(10)

,user_id,item_id,playtime_forever,playtime_2weeks
305371,TheSuccIsGudd,207140,18.0,0.0
852792,mutatedwombat,70,36.0,0.0
426006,76561198087570173,242550,497.0,0.0
262301,sapperau,31280,828.0,0.0
1064005,76561198067243010,346900,1218.0,0.0
575034,djnemonic,337950,8.0,0.0
68109,chidvd,244090,519.0,0.0
1048409,TaylorSinclair,236430,6759.0,0.0
490512,76561198095920833,203140,659.0,0.0
779689,lelvis,15620,348.0,0.0


In [ ]:
steam_games.rename(columns={'id': 'item_id'}, inplace=True) # Renombrar columnas

In [ ]:
# Modificar los tipos de datos de los dataframe

steam_games['item_id'] = steam_games['item_id'].astype('Int64')
df_items['item_id'] = df_items['item_id'].astype('Int64')
df_reviews['item_id'] = df_reviews['item_id'].astype('Int64')

In [ ]:
steam_games_items = pd.merge(steam_games,df_items, on='item_id', how='inner') # Merge los datos de los games con los datos de los items
steam_games_items.to_csv('steam_games_items.csv', index=False) # Guarda los datos en un archivo csv

In [ ]:
steam_games_review = pd.merge(steam_games,df_reviews, on='item_id', how='inner') # Merge los datos de los games con los datos de los items
steam_games_review.to_csv('steam_games_review.csv') # Guardar los datos en un csv